# Example for Qwen3-VL-Embedding using vLLM

## 1. Import Packages and Prepare Utility Functions

In [1]:
import numpy as np
import os
from typing import List, Dict, Any
from vllm import LLM
from vllm.multimodal.utils import fetch_image
from PIL import Image

def format_input_to_conversation(
    input_dict: Dict[str, Any], 
    default_instruction: str = "Represent the user's input."
) -> List[Dict]:
    content = []
    
    instruction = input_dict.get('instruction') or default_instruction
    text = input_dict.get('text')
    image = input_dict.get('image')
    
    if image:
        image_content = None
        if isinstance(image, str):
            if image.startswith(('http://', 'https://')):
                image_content = image
            else:
                abs_image_path = os.path.abspath(image)
                image_content = 'file://' + abs_image_path
        else:
            image_content = image
        
        if image_content:
            content.append({
                'type': 'image', 
                'image': image_content,
            })
    
    if text:
        content.append({'type': 'text', 'text': text})
    
    if not content:
        content.append({'type': 'text', 'text': ""})
    
    conversation = [
        {"role": "system", "content": [{"type": "text", "text": instruction}]},
        {"role": "user", "content": content}
    ]
    
    return conversation

def prepare_vllm_inputs(
    input_dict: Dict[str, Any], 
    llm, 
) -> Dict[str, Any]:
    conversation = format_input_to_conversation(input_dict)
    
    prompt_text = llm.llm_engine.tokenizer.apply_chat_template(
        conversation, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    multi_modal_data = None
    image = input_dict.get('image')
    if image:
        if isinstance(image, str):
            if image.startswith(('http://', 'https://')):
                try:
                    image_obj = fetch_image(image)
                    multi_modal_data = {"image": image_obj}
                except Exception as e:
                    print(f"Warning: Failed to fetch image {image}: {e}")
            else:
                abs_image_path = os.path.abspath(image)
                if os.path.exists(abs_image_path):
                    image_obj = Image.open(abs_image_path)
                    multi_modal_data = {"image": image_obj}
                else:
                    print(f"Warning: Image file not found: {abs_image_path}")
        else:
            multi_modal_data = {"image": image}
    
    result = {
        "prompt": prompt_text,
        "multi_modal_data": multi_modal_data
    }
    return result

/cpfs01/user/linqi.lmx/envs/vllm014n/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Convert Inputs

In [ ]:
# Initialize model
llm = LLM(
    model="Qwen/Qwen3-VL-Embedding-2B",
    runner="pooling",
    dtype='bfloat16',
    trust_remote_code=True,
)

# Prepare input samples
inputs = [
    {
        "text": "A woman playing with her dog on a beach at sunset.",
        "instruction": "Retrieve images or text relevant to the user's query.",
    },
    {
        "text": "A woman shares a joyful moment with her golden retriever on a sun-drenched beach at sunset, as the dog offers its paw in a heartwarming display of companionship and trust."
    },
    {
        "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
    },
    {
        "text": "A woman shares a joyful moment with her golden retriever on a sun-drenched beach at sunset, as the dog offers its paw in a heartwarming display of companionship and trust.",
        "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
    }
]

print(f"Prepared {len(inputs)} input samples")

# Convert to vLLM format
vllm_inputs = [prepare_vllm_inputs(inp, llm) for inp in inputs]

print("Input conversion completed!")
print(f"\nPreview of the first input prompt:")
print(vllm_inputs[0]["prompt"][:200] + "...")

INFO 01-16 07:47:29 [utils.py:267] non-default args: {'runner': 'pooling', 'trust_remote_code': True, 'dtype': 'bfloat16', 'disable_log_stats': True, 'model': '/cpfs01/user/linqi.lmx/models/finetune/qwen/qwen3-vl/Qwen3-VL-Embedding/Qwen3-VL-Embedding-2B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 01-16 07:47:29 [model.py:859] Resolved `--convert auto` to `--convert embed`. Pass the value explicitly to silence this message.


INFO 01-16 07:47:29 [model.py:530] Resolved architecture: Qwen3VLForConditionalGeneration


...

(EngineCore_DP0 pid=82232) 

INFO 01-16 07:48:14 [core.py:272] init engine (profile, create kv cache, warmup model) took 27.63 seconds


INFO 01-16 07:48:21 [llm.py:347] Supported tasks: ['embed', 'token_embed']


Prepared 4 input samples


Input conversion completed!

Preview of the first input prompt:
<|im_start|>system
Retrieve images or text relevant to the user's query.<|im_end|>
<|im_start|>user
A woman playing with her dog on a beach at sunset.<|im_end|>
<|im_start|>assistant
...


## 3. Get Embeddings

In [3]:
# Generate embeddings
outputs = llm.embed(vllm_inputs)

# Extract embedding vectors
embeddings_list = []
for i, output in enumerate(outputs):
    emb = output.outputs.embedding
    embeddings_list.append(emb)

embeddings = np.array(embeddings_list)
print(f"Embeddings shape: {embeddings.shape}")
print(f"Embedding dimension per sample: {embeddings.shape[1]}")


Adding requests:   0%|                                                                                                                                                    | 0/4 [00:00<?, ?it/s]


Adding requests:  75%|█████████████████████████████████████████████████████████████████████████████████████████████████████████                                   | 3/4 [00:02<00:00,  1.14it/s]


Adding requests: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.50it/s]


Processed prompts:   0%|                                                                                              | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 53.65it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 53.33it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Embeddings shape: (4, 2048)
Embedding dimension per sample: 2048


## 4. Display Similarity Scores

In [4]:
# Calculate similarity matrix
similarity_scores = embeddings @ embeddings.T

print("Similarity Score Matrix:")
print(similarity_scores)

Similarity Score Matrix:
[[1.00000007 0.78962161 0.71557983 0.72364763]
 [0.78962161 0.9999999  0.76502916 0.84728952]
 [0.71557983 0.76502916 0.99999998 0.82388284]
 [0.72364763 0.84728952 0.82388284 1.00000003]]
